## CNN
    
    

In [1]:
 # Define transformations (resize and convert to tensor)

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
    ])

    # Define dataset directories
train_dir = "../data/Training"
test_dir = "../data/Testing"
valid_dir = "../data/Validation"

    # Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

    # Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Print class-to-index mapping
print("Class mapping:", train_dataset.class_to_idx)

    # Check a batch of images and labels
images, labels = next(iter(train_loader))
print(f"Batch image shape: {images.shape}")
print(f"Batch labels: {labels}")

    # Checking the label counts
label_counts = Counter(train_dataset.targets)
print("Training class distribution:", label_counts)

label_counts = Counter(test_dataset.targets)
print("Testing class distribution:", label_counts)  

label_counts = Counter(valid_dataset.targets)
print("Validation class distribution:", label_counts)  
    

Class mapping: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
Batch image shape: torch.Size([32, 3, 224, 224])
Batch labels: tensor([3, 3, 1, 3, 1, 1, 3, 2, 3, 0, 2, 1, 1, 3, 0, 1, 0, 0, 3, 3, 0, 2, 2, 1,
        0, 0, 1, 3, 0, 2, 0, 3])
Training class distribution: Counter({3: 827, 0: 826, 1: 822, 2: 395})
Testing class distribution: Counter({1: 58, 0: 50, 3: 37, 2: 27})
Validation class distribution: Counter({1: 57, 0: 50, 3: 37, 2: 26})


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        
        # this function defines the nn structure
        super().__init__()
        self.cnn_model = nn.Sequential(

            # conv layer 1
            # 1 input as each pixel is level of gray
            nn.Conv2d(in_channels = 3, out_channels=8, kernel_size=3),

            # normalize batch to reduce overfitting
            nn.BatchNorm2d(8),

            #Relu activation, more commonly used than tanh
            nn.ReLU(),

            nn.Dropout(0.3),

            # pool elements
            # might want to do kernel_size of 3
            nn.MaxPool2d(kernel_size = 3, stride = 3),

            # conv layer 2
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size = 3),

             # normalize batch to reduce overfitting
            nn.BatchNorm2d(16),

            #Relu activation, more commonly used than tanh
            nn.ReLU(),

            nn.Dropout(0.3),

            # pool elements
            # might want to do kernel_size of 3
            nn.MaxPool2d(kernel_size = 3, stride = 4),
        )

        self.dense_model = nn.Sequential(
            # adjust in_features based on performance
            # first linear layer
            nn.Linear(in_features=5184, out_features = 512),
            nn.ReLU(),

            # add droput layer to help with overfitting
            nn.Dropout(0.4),

            # second
            #nn.Linear(in_features=128, out_features = 64),
            #nn.ReLU(),
            nn.Linear(in_features=512, out_features = 4),
            # softmax is already applied for cross entropy loss 
        )

    def forward(self, x):
        # this function defines how the information goes through the nn

        # send through cnn
        x = self.cnn_model(x)

        # flatten output
        x = x.view(x.size(0), -1)

        # send through dense 
        x = self.dense_model(x)

        return x

# Get number of classes

# Initialize the model
model = CNN()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_counts = torch.tensor([826, 822, 395, 827])  # Your class distribution
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()  # Normalize

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

## Testing

In [3]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Ensure model is on the correct device

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Accuracy calculation
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct / total * 100
        
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

# Train for 25 epochs
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=25)

Epoch 1, Train Loss: 1.5074, Val Loss: 1.5703, Val Acc: 35.88%
Epoch 2, Train Loss: 0.6277, Val Loss: 1.7190, Val Acc: 38.24%
Epoch 3, Train Loss: 0.5145, Val Loss: 1.6692, Val Acc: 41.18%
Epoch 4, Train Loss: 0.4484, Val Loss: 1.8906, Val Acc: 41.76%
Epoch 5, Train Loss: 0.3564, Val Loss: 1.6698, Val Acc: 49.41%
Epoch 6, Train Loss: 0.3557, Val Loss: 1.5208, Val Acc: 58.24%
Epoch 7, Train Loss: 0.3385, Val Loss: 1.5927, Val Acc: 52.94%
Epoch 8, Train Loss: 0.2545, Val Loss: 1.9415, Val Acc: 55.88%
Epoch 9, Train Loss: 0.1972, Val Loss: 1.8772, Val Acc: 52.94%
Epoch 10, Train Loss: 0.1813, Val Loss: 1.9760, Val Acc: 50.59%
Epoch 11, Train Loss: 0.1957, Val Loss: 2.2870, Val Acc: 55.29%
Epoch 12, Train Loss: 0.1833, Val Loss: 2.1482, Val Acc: 62.94%
Epoch 13, Train Loss: 0.1693, Val Loss: 2.3508, Val Acc: 58.82%
Epoch 14, Train Loss: 0.1612, Val Loss: 2.0125, Val Acc: 68.82%
Epoch 15, Train Loss: 0.1248, Val Loss: 2.1295, Val Acc: 67.65%
Epoch 16, Train Loss: 0.1103, Val Loss: 1.9072, V

In [4]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Evaluate the model
evaluate_model(model, test_loader)
evaluate_model(model, train_loader)


Test Accuracy: 67.44%


In [5]:
import torch

def calculate_recall_without_sklearn(model, data_loader):
    model.eval()
    true_positives = torch.zeros(4)
    false_negatives = torch.zeros(4)
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for i in range(4):
                true_positives[i] += ((predicted == i) & (labels == i)).sum().item()
                false_negatives[i] += ((predicted != i) & (labels == i)).sum().item()
    
    # Compute recall per class
    recall = true_positives / (true_positives + false_negatives)
    recall[torch.isnan(recall)] = 0  # Handle division by zero cases

    # Convert to dictionary
    recall_dict = {f"Class {i}": recall[i].item() for i in range(4)}

    return recall_dict

# Example usage:
recall_rates = calculate_recall_without_sklearn(model, test_loader)
print("Recall per class:", recall_rates)

Recall per class: {'Class 0': 0.20000000298023224, 'Class 1': 0.6896551847457886, 'Class 2': 1.0, 'Class 3': 0.9459459185600281}


In [6]:
def compute_confusion_matrix(model, data_loader):
    model.eval()
    confusion_matrix = torch.zeros(4, 4, dtype=torch.int64)

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t, p] += 1  # Increment the matrix count

    return confusion_matrix

# Example usage:
conf_matrix = compute_confusion_matrix(model, test_loader)
test_conf = compute_confusion_matrix(model, train_loader)

def print_confusion_matrix(conf_matrix, class_names):
    print("\nConfusion Matrix:")
    print(" " * 10, end="")  # Formatting for column headers
    print("  ".join(f"{cls[:5]:>5}" for cls in class_names))  # Print class names as column headers

    for i, row in enumerate(conf_matrix):
        print(f"{class_names[i][:5]:>10} ", end="")  # Row labels
        print("  ".join(f"{val:5}" for val in row.tolist()))  # Print matrix row values

# Example usage:
print_confusion_matrix(conf_matrix, train_dataset.classes)
print_confusion_matrix(test_conf, train_dataset.classes)


Confusion Matrix:
          gliom  menin  no_tu  pitui
     gliom    10      6     26      8
     menin     3     40      2     13
     no_tu     0      0     27      0
     pitui     0      2      0     35

Confusion Matrix:
          gliom  menin  no_tu  pitui
     gliom   826      0      0      0
     menin    29    763      3     27
     no_tu     0      0    394      1
     pitui     0      0      0    827
